In [0]:
!pip install -q keras

In [0]:
from google.colab import files
uploaded = files.upload()

Saving testx_final.npy to testx_final.npy
Saving testy_final.npy to testy_final.npy
Saving trainx_final.npy to trainx_final.npy
Saving trainy_final.npy to trainy_final.npy
Saving validx_final.npy to validx_final.npy
Saving validy_final.npy to validy_final.npy


In [0]:
for k in uploaded.keys():
  with open(k, 'wb') as f:
    f.write(uploaded[k])

In [0]:
!ls

sample_data	 testy_final.npy   trainy_final.npy  validy_final.npy
testx_final.npy  trainx_final.npy  validx_final.npy


In [0]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Activation
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K

Using TensorFlow backend.


In [0]:
K.set_image_dim_ordering('th')

# Random seed
np.random.seed(1)

# Load data
print("Loading data files")
X_valid = np.load("validx_final.npy")
X_train = np.load("trainx_final.npy")
X_test = np.load("testx_final.npy")

y_valid = np.load("validy_final.npy")
y_train = np.load("trainy_final.npy")
y_test = np.load("testy_final.npy")

# Sanity check
print("Dataset Shape: {0}: ".format(X_train.shape))
seq_length = len(X_valid[0][0])
print("Sequence Length: {0}".format(seq_length))

# One hot Encoding
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

Loading data files
Dataset Shape: (216000, 5, 24): 
Sequence Length: 24


In [0]:
#X_train = X_train[:,:,0:15]
#X_valid = X_valid[:,:,0:15]
#X_test = X_test[:,:,0:15]

In [0]:
'''
# Keras Conv1D Example Model 

model = Sequential()
model.add(Conv1D(64, 3, activation='relu', data_format='channels_first', input_shape=(5, seq_length)))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=16, epochs=10)
score = model.evaluate(X_test, y_test, batch_size=16)

# Final model evaluation
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1] * 100))

'''

# Create the model
model = Sequential()
model.add(Conv1D(32, 3, input_shape=(5, 24), data_format='channels_first', padding='same', activation='relu'))
model.add(Dropout(0.2))
model.add(Conv1D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, 3, activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv1D(64, 3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(128, 3, activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(2))
#model.add(Activation("softmax"))

# Compile model
epochs = 25
lrate = 0.01
decay = lrate / epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1] * 100))

'''
def generator(inputs, labels):
    i = 0

    while True:

        index = inputs[i % len(inputs)]
        inputs_batch = np.array([(index[0]), (index[1]), (index[2]), (index[3]), (index[4])])
        inputs_batch = np.expand_dims(inputs_batch, axis=0)

        #print("\nNEW inputs_batch: {0}".format(inputs_batch))
        #print(inputs_batch.shape)

        labels_batch = np.array([labels[i % len(inputs)]])
        yield inputs_batch, labels_batch
        i += 1
        
        
# Fit the model using a Generator
model.fit_generator(
    generator(X_train, y_train),
    validation_data=generator(X_valid, y_valid),
    steps_per_epoch=len(X_train),
    validation_steps=len(X_valid),
    epochs=epochs
)

'''

# Final model evaluation
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1] * 100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 32, 24)            512       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 24)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 32, 32)            2336      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 16, 32)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 16, 64)            6208      
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 64)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 16, 64)            12352     
__________